In [ ]:
from tbh import runner_tools as rt 
from tbh import plotting as pl

from importlib import reload
reload(rt);
reload(pl);

In [ ]:
from tbh import full_model as fm 

reload(fm)

model_config = {    
    "start_time": 1800,
    "end_time": 2025,
    "seed": 100
}

model = fm.get_tb_model(model_config)

In [ ]:
model.get_input_parameters()

In [ ]:
params = {
    "init_pop_size": 1.e6,
    "raw_transmission_rate": 10.,

    # Susceptibility to infection
    'rel_sus_mtb_naive': 1., # !FIXME we don't need a parameter for this...
    'rel_sus_cleared': 1.,
    'rel_sus_contained': 0.2,
    'rel_sus_recovered': 1.,

    # Early TB infection
    'breakdown_rate': .1,
    'clearance_rate': .1,
    'containment_rate': 5.,
    'progression_rate': .5,

    # Active TB dynamics
    'clinical_progression_rate': 1.,
    'clinical_regression_rate': 1.,
    'infectiousness_gain_rate': 1.,
    'infectiousness_loss_rate': 1.,
}

model.run(params)

In [ ]:
model.flows

In [ ]:
df = model.get_derived_outputs_df()

In [ ]:
df['raw_incidence'].plot()

In [ ]:
studies_dict = rt.DEFAULT_STUDIES_DICT
params = rt.get_full_default_params(studies_dict)

# Manual run

In [ ]:
model, do_df = rt.model_single_run(rt.DEFAULT_MODEL_CONFIG, studies_dict, params)

In [ ]:
do_df[['ltbi_propXmajuro']].plot()